In [1]:
# !wget https://raw.githubusercontent.com/attardi/wikiextractor/master/WikiExtractor.py
# !wget http://dumps.wikimedia.org/mswiki/latest/mswiki-latest-pages-articles.xml.bz2
# !bzip2 -d mswiki-latest-pages-articles.xml.bz2
# !python3 WikiExtractor.py --json mswiki-latest-pages-articles.xml

In [2]:
from glob import glob

files = glob('text/*/*')
len(files)

249

In [3]:
import json

with open(files[0]) as fopen:
    x = fopen.readlines()

In [4]:
import re
from unidecode import unidecode

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

def split_into_sentences(text):
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('\s' + alphabets + '[.] ', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.] ' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [cleaning(s.strip()) for s in sentences if len(s) > 10]
    return sentences

In [5]:
split_into_sentences(json.loads(x[0])['text'].replace('<br>', ' '))

['Arif Shah Omar Shah Datuk Arif Shah Haji Omar Shah (lahir 2 November 1956) merupakan bekas Ahli Dewan Undangan Seberang Jaya, Pulau Pinang selama dua penggal berturut-turut sejak 1999.',
 'Beliau dilahirkan pada dan merupakan anak ke-4 daripada keseluruhan 10 orang adik beradik.',
 'Mengikut Jeff Ooi, penulis blog yang juga Ahli Parlimen Jelutong, beliau merupakan adik kepada Tan Sri Amin Shah Omar Shah, bekas seorang ahli perniagaan Malaysia yang suatu masa dulu terkenal dengan industri perkapalan.',
 'Beliau digelar oleh masyarakat Cina setempat sebagai "pek moh" (Bai Mao ), bermaksud "bulu putih" dalam Bahasa Hokkien, kerana beliau penuh dengan rambut putih.',
 'Datuk Arif Shah adalah merupakan anak keempat daripada 9 orang adik beradik.',
 'Adik beradik terdiri daripada nama-nama berikut:- 1.',
 'Puan Fadzilah Omar Shah 2.',
 'Tan Sri Amin Shah Omar Shah 3.',
 "Dato' Faridah Omar Shah 4.",
 '"Datuk Arif Shah Omar Shah PJK DMSM" 5.',
 'Datuk Azman Shah Omar Shah 6.',
 'Tuan Azhar 

In [6]:
from tqdm import tqdm

texts = []

for i in tqdm(range(len(files))):
    with open(files[i]) as fopen:
        s = fopen.readlines()
    
    for i in s:
        i = json.loads(i)
        texts.extend([i['title']] + split_into_sentences(i['text'].replace('<br>', ' ')))
        texts.append('')

100%|██████████| 249/249 [01:52<00:00,  2.22it/s]


In [ ]:
with open('wiki-text.txt', 'w') as fopen:
    fopen.write('\n'.join(texts))